# core

> Fill in a module description here

In [1]:
#| default_exp concepts

In [2]:
#| export
import numpy as np

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
num_steps = 100  # number of points in timeseries
num_samples = 2  # number of time series
num_features = 1  # features measured per time step

In [5]:
np.random.seed(42)

ts_sample = np.random.randn(num_steps, num_features)  # a single sample from ts
# ts_samples = np.random.randn(num_samples, num_steps, num_features)  # multiple ts samples 

ts_label = np.random.choice([0, 1], p=[0.95, 0.05], size=num_steps)  # 1 is the anomaly
ts_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

Goal is to predict if an anomaly occured at time step `t`.

In [6]:
p = 10  # number of past consecutive readings in a window -> encodes into q (latent space dim)
end_time = 100
start_time = end_time - p 
f"time series index starting at {start_time} and ending at {end_time} has label {ts_label[end_time-1]}"

'time series index starting at 90 and ending at 100 has label 0'

In [15]:
#| export 

def get_window(x, window_size=10, end_step=100, indices=None, return_indices=True):
    """
    Returns a window from x of window_size, ending in end_step.

    If actual indices are passed, a window corresponding to that will be taken.
    """
    start_step = end_step-window_size
    indices = np.asarray(range(0, len(x))) if indices is None else indices
    if return_indices:
        return indices[start_step:end_step]
    else:
        return x[indices[start_step:end_step], :]  # x of shape (num_features, feature_len)

In [16]:
ts_sample.shape

(100, 1)

In [17]:
get_window(ts_sample, window_size=p, end_step=num_steps, return_indices=True)

array([90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

There can be multiple sequences ending at time t based the `p` we set. 

Maximum possible rolling windows, for all possible `end_time`: `num_steps-p`

VAEs will be trained on such a "rolling window".

In [18]:
max_windows = num_steps-p
max_windows

90

In [19]:
get_window(ts_sample, window_size=p, end_step=10, return_indices=True)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

LSTM is trained on the embeddings generated by the VAE encoder.

In [20]:
k=10  # non overlapping window size

When `k=1`, then the windows will overlap.

In [21]:
for time in range(10, 100, k):
    # assuming we just want windows at every 10th step
    indices = get_window(ts_sample, window_size=p, end_step=time)
    start_time = indices[0]
    print(f"time series index starting at {start_time} and ending at {indices[-1]} has label {ts_label[time]}")

time series index starting at 0 and ending at 9 has label 1
time series index starting at 10 and ending at 19 has label 0
time series index starting at 20 and ending at 29 has label 0
time series index starting at 30 and ending at 39 has label 0
time series index starting at 40 and ending at 49 has label 0
time series index starting at 50 and ending at 59 has label 0
time series index starting at 60 and ending at 69 has label 0
time series index starting at 70 and ending at 79 has label 0
time series index starting at 80 and ending at 89 has label 0


- All of these windows of varying lengths can be embdedded into the same dimension `q`. 

- The LSTM model acts on these embeddings, but those embeddings should correspond to non-overlapping windows.

> I guess, for each `end_time`, there can only be one such window.

- After the VAE model has been optimised, we use the encoder from the trained VAE model to estimate the embedding sequences $E_t$.

- To train the LSTM model, we have the LSTM model take the first `k − 1` embeddings in a sequence $E_t$ and predict the next `k − 1` embeddings.

- (most important) All the model parameters for both VAE and LSTM units are optimised without anomaly labels
- 
- Also need to define a threshold $\theta$ on the score function $d_t$, above which we will flag an anomaly alert $y_t = 1$ at the current $t$.

In [22]:
#| hide
import nbdev; nbdev.nbdev_export()